<div style="font-size: 200%; font-weight: bold; color: maroon;">Procesado de la red de vuelos con NetworkX (I)</div>
<!-- Paulo Villegas, 2020 - v. 2.2 -->

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pyplot as mpl
import networkx as nx

%matplotlib inline
plt.style.use('bmh')
mpl.rcParams['figure.figsize'] = (16, 10)

# Datos

Lee datos de ficheros CSV a DataFrames de Pandas

In [ ]:
airports = pd.read_csv('DATA/openflights/airports.csv.bz2', escapechar="\\", na_values='\\N')
routes = pd.read_csv('DATA/openflights/routes.csv.bz2', na_values='\\N')

In [ ]:
len(airports)

In [ ]:
len(routes)

In [ ]:
airports.columns

Limpiamos el grafo quitando las rutas para las que no tenemos aeropuerto

In [ ]:
airport_set = set(airports.IATA)

In [ ]:
invalid = ~routes.SourceAirport.isin(airport_set) | ~routes.DestAirport.isin(airport_set)
routes_clean = routes.drop( routes[invalid].index )

In [ ]:
len(routes_clean)

# Red

Crea la red:

* usamos el DataFrame de rutas (los enlaces) para crear directamente el grafo. Basta indicar las columnas que corresponden al nodo de origen y de destino, y columnas adicionales que queramos añadir como atributos del enlace
* añadimos los datos sobre aeropuertos como atributos de los nodos del grafo

In [ ]:
# Crea la red desde el DataFrame de enlace. Aparte de origen & destino, añadimos 3 columnas como atributos
vuelos = nx.from_pandas_edgelist(routes_clean,'SourceAirport','DestAirport', ['Airline', 'Codeshare', 'Stops'])

# Añadimos atributos de los nodos iterando sobre el DataFrame de los aeropuertos 
# (el id del nodo es el código IATA del aeropuerto)
for ap in airports.itertuples(index=False):
    vuelos.add_node(ap.IATA, **ap._asdict())

# Representación y cálculos

## Grafos
Vamos a visualizar el grafo con dos métodos distintos

**[1]** Usamos primero la conversión directa desde NetworkX a matplotlib

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 6))
%time nx.draw_networkx(vuelos, ax=ax, with_labels=False, node_size=5, width=.5)
ax.set_axis_off()

**[2]** Ahora creamos una función para convertir de NetworkX a GraphViz (usando PyDot) y luego desde GraphViz a PNG

In [ ]:
from IPython.display import Image

def render(g, engine="neato", **kwargs):
    '''
     :param g (Graph): the graph to render
     :param engine (str): the GraphViz engine to use
     :param kwargs (dict): formatting options for nodes
    '''
    node_prop = {'style': 'filled', 'shape': 'circle'}
    node_prop.update(kwargs)   # add any passed node properties
    # Define default graph, edge & node properties
    g.graph['graph'] = {'dpi': 72, 'size': '12,8!', 'ratio': 'expand'}
    g.graph['edge'] = {'color': '#40e0d0d0'}
    g.graph['node'] = node_prop
    # Convert to GraphViz
    dot = nx.nx_pydot.to_pydot(g)
    # Render into SVG, and embed into HTML
    svg = dot.create_png(prog=engine)
    return Image(svg)

In [ ]:
%time render(vuelos, 'sfdp')

## Componente principal

In [ ]:
# Calcula todos los componentes conectados, y los ordena por tamaño
cc = sorted(nx.connected_components(vuelos), key=len, reverse=True)

In [ ]:
bigc = vuelos.subgraph(cc[0])

In [ ]:
len(bigc)

In [ ]:
%time render(bigc, "sfdp")

## Componentes conectados

In [ ]:
cclist = [(a, cnum+1) for cnum, nodelist in enumerate(cc) for a in nodelist]

df = pd.DataFrame(cclist, columns=('airportId', 'cc'))

In [ ]:
df.groupby('cc').count().reset_index()

In [ ]:
i = df.loc[df['cc']==3].sort_values('airportId')['airportId']
i

In [ ]:
airports[airports.IATA == 'ESD']

In [ ]:
airports[airports.IATA.isin(set(i))]

## Mapa

Código adaptado de [Drawing flight routes with NetworkX](http://ipython-books.github.io/142-drawing-flight-routes-with-networkx/)

In [ ]:
import cartopy.crs as ccrs

In [ ]:
# Diccionario con la posición de cada aeropuerto
pos = {a.IATA: (a.Longitude, a.Latitude)
       for a in airports.itertuples()}

In [ ]:
# tamaño de cada nodo
from math import log10
deg = nx.degree(bigc)
bigc_sizes = [4*log10(deg[iata])+1 for iata in bigc.nodes]

In [ ]:
crs = ccrs.PlateCarree()
fig, ax = plt.subplots(figsize=(22, 16), subplot_kw=dict(projection=crs))
ax.coastlines()

# Rotula solo los nodos con grado alto
labels = {iata: iata if deg[iata] >= 30 else ''
          for iata in bigc.nodes}

nx.draw_networkx(bigc, ax=ax,
                 font_size=10,
                 alpha=.8,
                 width=.075,
                 node_size=bigc_sizes,
                 labels=labels,
                 pos=pos,
                 cmap=plt.cm.autumn)

## Subgrafos

Como ejemplo de un subgrafo del grafo completo, creamos la red de vuelos de Iberia

In [ ]:
routes_ib = routes_clean[routes_clean.Airline == 'IB']

In [ ]:
airports_ib_list = set(routes_ib.SourceAirport) | set(routes_ib.DestAirport)
airports_ib = airports[airports.IATA.isin(airports_ib_list)]

In [ ]:
vuelos_ib = nx.from_pandas_edgelist(routes_ib,'SourceAirport','DestAirport')
for ap in airports_ib.itertuples(index=False):
    vuelos_ib.add_node(ap.IATA, **ap._asdict())

In [ ]:
import cartopy.crs as ccrs
from math import log10

crs = ccrs.PlateCarree()
fig, ax = plt.subplots(figsize=(20, 14), subplot_kw=dict(projection=crs))
ax.coastlines()
ax.stock_img()

pos_ib = {a.IATA: (a.Longitude, a.Latitude)
           for a in airports_ib.itertuples()}
deg_ib = nx.degree(vuelos_ib)
sizes = [10*log10(deg_ib[iata])+1 for iata in vuelos_ib.nodes]
labels = {iata: iata if deg_ib[iata] >= 30 else ''
          for iata in vuelos_ib.nodes}

nx.draw_networkx(vuelos_ib, ax=ax,
                 font_size=6,
                 alpha=.5,
                 width=.25,
                 node_size=sizes,
                 labels=labels,
                 pos=pos_ib,
                 #node_color=altitude,
                 cmap=plt.cm.autumn
                )

La misma red, pero con la visualización concentrada en Europa 

In [ ]:
import cartopy

sizes = [20*deg_ib[iata]+10 for iata in vuelos_ib.nodes]
labels = {iata: iata if deg_ib[iata] >= 15 else ''
          for iata in vuelos_ib.nodes}

fig, ax = plt.subplots(1, 1, figsize=(20, 18))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_extent([-20, 30, 30, 80])
ax.coastlines(resolution='50m')
ax.gridlines();

# Low-resolution country borders & ocean fill
#ax.add_feature(cartopy.feature.BORDERS, linestyle='-', alpha=.5)
#ax.add_feature(cartopy.feature.OCEAN, facecolor=(0.2,0.9,1.0), alpha=.8)

# Higher resolution features
import cartopy.feature as cfeature
borders_50m = cfeature.NaturalEarthFeature('cultural', 'admin_0_boundary_lines_land', '50m',
                                           edgecolor='green', facecolor='none')
ocean_50m = cfeature.NaturalEarthFeature('physical', 'ocean', '50m',
                                         edgecolor='face', linestyle='--', facecolor=cfeature.COLORS['water'])
ax.add_feature(ocean_50m, zorder=-1)
ax.add_feature(borders_50m, zorder=-1)

nx.draw_networkx(vuelos_ib, ax=ax,
                 font_size=20, alpha=.8, width=.25,
                 node_color="sienna",
                 node_size=sizes,
                 pos=pos_ib,
                 labels=labels,
                 with_labels=True,
                 cmap=plt.cm.autumn)